# Prepare Downloaded IPL Data

Data Source: https://cricsheet.org/downloads/

Data Information: https://cricsheet.org/format/csv_ashwin/#introduction-to-the-quot-ashwin-quot-csv-format

In [5]:
# Import required packages
import pandas as pd
import numpy as np
import os

# Prepare IPL Match Info data

In [6]:
# Function to store file temporary
def get_files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [7]:
# Get the list of file names from directory
folder_name = 'ipl_csv2'
list_files = []
for file in get_files(folder_name):
    list_files.append(file)


# Get the _info files from the from directory
list_info_files = []
list_match_id = []
for ind, file in enumerate(list_files):
    if file[-9:] == '_info.csv':
        list_info_files.append(file)
        list_match_id.append(file[:-9])

In [8]:
def get_match_info_data(list_filenames, dir_name, list_id):
    match_info_list_dict = []
    for i in range(len(list_filenames)):
        dict = {'match_id': list_id[i]}
        df = pd.read_csv(str(dir_name)+'/'+str(list_filenames[i]), on_bad_lines='skip')
        col = list(df.iloc[1:,0])
        data = list(df.iloc[1:,1])
        for i in range(len(col)):
            if col[i] in dict:
                dict[col[i]+'_1'] = data[i]
            else:
                dict[col[i]] = data[i]
#         dict = {key_cols[ind]: data[ind] for ind in range(len(key_cols))}
        match_info_list_dict.append(dict)
    return match_info_list_dict

In [9]:
# Export data from csv and save in dataframe
match_info_dict = get_match_info_data(list_info_files, folder_name, list_match_id)
df = pd.DataFrame(match_info_dict)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   match_id         950 non-null    object
 1   team             950 non-null    object
 2   team_1           950 non-null    object
 3   gender           950 non-null    object
 4   season           950 non-null    object
 5   date             950 non-null    object
 6   event            950 non-null    object
 7   match_number     892 non-null    object
 8   venue            950 non-null    object
 9   city             899 non-null    object
 10  toss_winner      950 non-null    object
 11  toss_decision    950 non-null    object
 12  player_of_match  946 non-null    object
 13  umpire           950 non-null    object
 14  umpire_1         950 non-null    object
 15  reserve_umpire   926 non-null    object
 16  tv_umpire        947 non-null    object
 17  match_referee    950 non-null    ob

In [11]:
df.head()

,match_id,team,team_1,gender,season,date,event,match_number,venue,city,...,reserve_umpire,tv_umpire,match_referee,winner,winner_wickets,winner_runs,outcome,eliminator,method,date_1
0,729305,Rajasthan Royals,Royal Challengers Bangalore,male,2014,2014/04/26,Indian Premier League,14,Sheikh Zayed Stadium,Abu Dhabi,...,K Srinath,RK Illingworth,AJ Pycroft,Rajasthan Royals,6,NaN,NaN,NaN,NaN,NaN
1,392199,Royal Challengers Bangalore,Kolkata Knight Riders,male,2009,2009/04/29,Indian Premier League,19,Kingsmead,Durban,...,ZTA Ndamane,SL Shastri,D Govindjee,Royal Challengers Bangalore,5,NaN,NaN,NaN,NaN,NaN
2,392198,Delhi Daredevils,Rajasthan Royals,male,2009,2009/04/28,Indian Premier League,18,SuperSport Park,Centurion,...,JD Cloete,DJ Harper,J Srinath,Rajasthan Royals,5,NaN,NaN,NaN,NaN,NaN
3,734021,Sunrisers Hyderabad,Royal Challengers Bangalore,male,2014,2014/05/20,Indian Premier League,46,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,...,A Nand Kishore,CK Nandan,RS Madugalle,Sunrisers Hyderabad,7,NaN,NaN,NaN,NaN,NaN
4,336010,Kolkata Knight Riders,Royal Challengers Bangalore,male,2007/08,2008/05/08,Indian Premier League,29,Eden Gardens,Kolkata,...,NaN,BR Doctrove,FM Engineer,Kolkata Knight Riders,NaN,5,NaN,NaN,NaN,NaN


In [12]:
# Update column names
df.rename(columns={'team': 'team1', 'team_1': 'team2', 'umpire':'umpire1', 'umpire_1':'umpire2'}, inplace=True)

# Create new column based on existing columns
df.loc[df['winner_wickets'].notnull(), 'result_type'] = 'chased'
df.loc[df['winner_runs'].notnull(), 'result_type'] = 'defended'
df.loc[df['outcome'] == 'tie', 'result_type'] = 'tie'
df.loc[df['outcome'] == 'no result', 'result_type'] = 'no result'

df.loc[df['winner_wickets'].isnull(), 'winner_wickets'] = 0
df.loc[df['winner_runs'].isnull(), 'winner_runs'] = 0
df['results'] = df['winner_wickets'].astype(int) + df['winner_runs'].astype(int)

# Update data type
df['date'] = pd.to_datetime(df.date)
df['match_id'] = df.match_id.astype(int)
df['winner_wickets'] = df.winner_wickets.astype(int)
df['winner_runs'] = df.winner_runs.astype(int)

In [13]:
columns = ['match_id', 'season', 'date', 'city', 'venue', 'team1', 'team2',
           'toss_winner', 'toss_decision', 'player_of_match', 
           'winner', 'winner_wickets', 'winner_runs', 'outcome', 'result_type', 'results', 
           'gender', 'event', 'match_number', 'umpire1', 'umpire2', 'reserve_umpire', 
           'tv_umpire', 'match_referee', 'eliminator', 'method', 'date_1']

# Save in df as ordered columns
df_final = df[columns]
df_final = df_final.sort_values(by='match_id', ascending=True).reset_index(drop=True)

In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   match_id         950 non-null    int64         
 1   season           950 non-null    object        
 2   date             950 non-null    datetime64[ns]
 3   city             899 non-null    object        
 4   venue            950 non-null    object        
 5   team1            950 non-null    object        
 6   team2            950 non-null    object        
 7   toss_winner      950 non-null    object        
 8   toss_decision    950 non-null    object        
 9   player_of_match  946 non-null    object        
 10  winner           932 non-null    object        
 11  winner_wickets   950 non-null    int64         
 12  winner_runs      950 non-null    int64         
 13  outcome          18 non-null     object        
 14  result_type      950 non-null    object   

In [15]:
df_final.head()

,match_id,season,date,city,venue,team1,team2,toss_winner,toss_decision,player_of_match,...,event,match_number,umpire1,umpire2,reserve_umpire,tv_umpire,match_referee,eliminator,method,date_1
0,335982,2007/08,2008-04-18,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,BB McCullum,...,Indian Premier League,1,Asad Rauf,RE Koertzen,VN Kulkarni,AM Saheba,J Srinath,NaN,NaN,NaN
1,335983,2007/08,2008-04-19,Chandigarh,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,MEK Hussey,...,Indian Premier League,2,MR Benson,SL Shastri,MSS Ranawat,RB Tiffin,S Venkataraghavan,NaN,NaN,NaN
2,335984,2007/08,2008-04-19,Delhi,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,MF Maharoof,...,Indian Premier League,3,Aleem Dar,GA Pratapkumar,NaN,IL Howell,GR Viswanath,NaN,NaN,NaN
3,335985,2007/08,2008-04-20,Mumbai,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,MV Boucher,...,Indian Premier League,5,SJ Davis,DJ Harper,SN Bandekar,AV Jayaprakash,J Srinath,NaN,NaN,NaN
4,335986,2007/08,2008-04-20,Kolkata,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,DJ Hussey,...,Indian Premier League,4,BF Bowden,K Hariharan,F Gomes,Asad Rauf,FM Engineer,NaN,NaN,NaN


In [ ]:
# Save file to current directory
# df_final.to_csv(r'/Users/robot/MachineLearning/Kaggle/cricket/ipl_match_info_data.csv', index=False)
# os.getcwd()

# IPL Ball-by-Ball data

In [2]:
all_match = pd.read_csv('ipl_csv2/all_matches.csv', low_memory=False)
all_match.sort_values(by='match_id', ascending=True).reset_index(drop=True)
all_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225954 entries, 0 to 225953
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                225954 non-null  int64  
 1   season                  225954 non-null  object 
 2   start_date              225954 non-null  object 
 3   venue                   225954 non-null  object 
 4   innings                 225954 non-null  int64  
 5   ball                    225954 non-null  float64
 6   batting_team            225954 non-null  object 
 7   bowling_team            225954 non-null  object 
 8   striker                 225954 non-null  object 
 9   non_striker             225954 non-null  object 
 10  bowler                  225954 non-null  object 
 11  runs_off_bat            225954 non-null  int64  
 12  extras                  225954 non-null  int64  
 13  wides                   7026 non-null    float64
 14  noballs             

In [3]:
all_match.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Save file to current directory
# all_match.to_csv(r'/Users/robot/MachineLearning/Kaggle/cricket/ipl_match_ball_by_ball_data.csv', index=False)
# os.getcwd()

'/Users/robot/MachineLearning/Kaggle/cricket'